# EEEN30131 Power System Analysis: Week 03 - Newton Raphson

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides examples of the applications of Newton's method.

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as you can explore many different variations of the power flow formulation, which will better prepare you for the exams.

## List of contents

- [Newton's method](#Newton's-method)
  - [Newton's example](#Newton's-example)
- [Three-bus network](#Three-bus-network)
  - [Three-bus normal example](#Three-bus-normal-example)
  - [Three-bus interactive example](#Three-bus-interactive-example)
- [Two-bus network](#Two-bus-network)
  - [Two-bus normal example](#Two-bus-normal-example)
  - [Two-bus interactive example](#Two-bus-interactive-example)
- [Four-bus network](#Four-bus-network)
- [Three-bus radial network](#Three-bus-radial-network)
- [Create your own example](#Create-your-own-example)

## Before we begin

This notebook builds on the materials that were presented during the last two weeks. Therefore, before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131:
  - Week 1 - Nodal analysis 
  - Week 2 - Power Flow Formulation 
  - Week 3 - Newton Raphson
- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

If this data notebook is being used in Jupyter lite, the folders where the python code that supports this notebook are stored, have to be enabled.

In [ ]:
from pathlib import Path
if Path.cwd().drive == '':
    a_dir = Path("dir")
    a_dir.mkdir(exist_ok=True)

The `ipywidgets` library is used to allow the creation of interactive case studies.

In [1]:
try:
    import ipywidgets as widgets
except:
    import micropip
    await micropip.install('ipywidgets')
    import ipywidgets as widgets
from ipywidgets import interact

Some figures will be created using the `matplotlib` library.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

We will import tools developed in week 01 and week 02.

In [3]:
import math
from Code.Wk1_EEEN30131 import get_Ybus
from Code.Wk2_EEEN30131 import get_Bus_Type, develop_PF_Equations
from Code.Wk3_EEEN30131 import develop_Jacobian, Newtons_Method, Visualize_Elec

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Newton's method

Newton's method can be summarized with the equation below. To solve it, we need an initial guess, the equation (or system of equations) and the derivative of the equation.

$$
x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}
$$

Let us create an equation, such as the following quadratic equation.

$$
f(x) = A + B x + Cx^2
$$

In [4]:
def fx(x, A=1, B=1, C=1):
    return A*x**2 + B*x + C

If we know the equation (as it is the case for power flows), we can differentiate it and apply Newton's method:

$$
f'(x) = B + 2Cx
$$

In [5]:
def fxprime (x, A=1, B=1, C=1):
    return 2*A*x + B

def Newton_Step_Algebraic(A=-1, B=3, C=4, x=0, Extra=None):
    Current_error = fx(x, A, B, C)/fxprime (x, A, B, C)
    x = x - Current_error

    return(x, abs(Current_error))

However, even if we do not know the equation, we can still get the differentials by using numerical approximations:

$$
f'(x) \approx \Delta f(x) = \lim_{\Delta x \to 0} \frac{f(x + \Delta x) - f(x)}{\Delta x}
$$

In [6]:
def Newton_Step_Numeric(A=-1, B=3, C=4, x=0, Deltax=0.0001):
    Deltaf = (fx(x+Deltax, A, B, C) - fx(x, A, B, C))/Deltax
    Current_error = fx(x, A, B, C)/Deltaf
    x = x - Current_error

    return(x, abs(Current_error))

To demonstrate the difference between the approaches, let us create a method to plot the function and each step of Newton's method:

In [7]:
def plot_Newton_Quadratic(A=-1, B=3, C=4, x=0, Plot_Begin=-5, Plot_End=5, mthd=[], Deltax=0.0001):
    plt.figure()
    X = np.arange(Plot_Begin, Plot_End, 0.1)
    Y = [A*x**2 + B*x + C for x in X]
    plt.plot(X, Y, label='Function')
    
    X2 = [x, x]
    Y2 = [0, fx(x, A, B, C)]
    Acceptable_Error = 0.0001
    Current_error = np.inf
    Number_Iterations = 0
    while Current_error > Acceptable_Error:
        Number_Iterations += 1
        (x, Current_error) = mthd(A, B, C, x, Deltax)
        X2.append(x)
        Y2.append(0)
        X2.append(x)
        Y2.append(fx(x, A, B, C))
    
    plt.plot(X2, Y2, label='Guesses')
    
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.grid()
    plt.show()

    return (x, Current_error, Number_Iterations)

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Newton's example

Let us now use an example.

In [8]:
#%matplotlib widget
Guess = widgets.FloatSlider(min=-5,max=5,step=0.1,value=1)
ui = widgets.VBox([Guess])
def NRAlg(Guess):
    A = -1
    B = 3
    C = 4
    x = Guess

    # Select range of the figure
    Plot_Begin = -5
    Plot_End = 5

    # Apply Newton's method (algebraic)
    (x, error, iter) = plot_Newton_Quadratic(A, B, C, x, Plot_Begin, Plot_End, Newton_Step_Algebraic)
    print('The result converged to ', x, 'after', iter, 'iterations')
    print('Estimated error', error)
out = widgets.interactive_output(NRAlg, {'Guess': Guess})
display(widgets.HBox([ui, out]))

Guess = widgets.FloatSlider(min=-5,max=5,step=0.1,value=1)
Error = widgets.FloatSlider(min=0.0001,max=1,step=0.0001,value=0.0001,continuous_update=False)
ui = widgets.VBox([Guess, Error])
def NRNum(Guess, Error):
    A = -1
    B = 3
    C = 4
    x = Guess

    # Select range of the figure
    Plot_Begin = -5
    Plot_End = 5

    # Apply Newton's method (Numeric)
    (x, error, iter) = plot_Newton_Quadratic(A, B, C, x, Plot_Begin, Plot_End, Newton_Step_Numeric, Error)
    print('The result converged to ', x, 'after', iter, 'iterations')
    print('Estimated error', error)
out = widgets.interactive_output(NRNum, {'Guess': Guess, 'Error': Error})
display(widgets.HBox([ui, out]))

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Three-bus network

### Three-bus normal example

The example presented above can be solved with the tools as follows:

<img src="Figures/Week02_3Bus.png" alt="Fig01" class="bg-primary" width="500px">

In [9]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]
Load = [
    [2, complex(1.5, 0.8)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1}
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₂ = V₂*V₁*

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Three-bus interactive example

<img src="Figures/Week02_3Bus.png" alt="Fig01" class="bg-primary" width="500px">

In [10]:
@interact
def Bus3(P = widgets.FloatSlider(min=0,max=2,step=0.01,value=1.5),
         Q = widgets.FloatSlider(min=0,max=2,step=0.01,value=0.8)):
    Connectivity = [
        [1, 2, complex(0, 0.10)],
        [1, 3, complex(0, 0.20)],
        [2, 3, complex(0, 0.25)]
    ]
    Load = [
        [2, complex(P, Q)]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 },
        {'Bus':3, 'P':1, 'V':1}
    ]

    # From Week 01
    Ybus = get_Ybus(Connectivity, True, False)

    # From Week 02
    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    # From Week 03
    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

interactive(children=(FloatSlider(value=1.5, description='P', max=2.0, step=0.01), FloatSlider(value=0.8, desc…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Two-bus network

### Two-bus normal example

Let us now try a different example, a 2-bus system:

![Week02_2Bus.png](Figures/Week02_2Bus.png)

In [11]:
Connectivity = [
    [1, 2, complex(0, 0.10)]
]
Load = [
    [2, complex(0.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 1 branches and 2 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
_______|___________|__________

Ybus = 
 [[ 0.-10.j -0.+10.j]
 [-0.+10.j  0.-10.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -0.5000   -0.5000

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂
P₁ = 1*V₂*10.0*sin(0-𝜃₂)
P₁ = -10.0000V₂sin(𝜃₂).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] 
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₂*B₂,₂*sin𝜃₂,₂
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁
-0.5 = V₂*1*10.0*sin(𝜃₂-0)
-0.5 = 10.0000V₂sin(𝜃₂).....[Implicit]

Q1:
Q₁ = V₁V₁[G₁,₁sin𝜃₁,₁-B₁,₁cos𝜃₁,₁] + V₁V₂[G₁,₂sin𝜃₁,₂-B₁,₂cos𝜃₁,₂] 
Q₁ = V₁*V₁*-B₁,₁*cos𝜃₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂
Q₁ = V₁²*-B₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂
Q₁ = 1²*--

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Two-bus interactive example

![Week02_2Bus.png](Figures/Week02_2Bus.png)

In [12]:
@interact
def Bus3(P = widgets.FloatSlider(min=0,max=2,step=0.01,value=0.5),
         Q = widgets.FloatSlider(min=-2,max=2,step=0.01,value=0.5),
         R = widgets.FloatSlider(min=0,max=0.2,step=0.01,value=0),
         X = widgets.FloatSlider(min=0.0,max=0.2,step=0.01,value=0.1)
        ):
    Connectivity = [
        [1, 2, complex(R, X)]
    ]
    Load = [
        [2, complex(P, Q)]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 }
    ]

    # From Week 01
    Ybus = get_Ybus(Connectivity, True, False)

    # From Week 02
    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    # From Week 03
    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

interactive(children=(FloatSlider(value=0.5, description='P', max=2.0, step=0.01), FloatSlider(value=0.5, desc…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Four-bus network

Let us now try a bigger network:

![Week02_4Bus.png](Figures/Week02_4Bus.png)

In [13]:
Connectivity = [
    [1, 2, complex(0, 0.25)],
    [1, 3, complex(0, 0.5)],
    [3, 4, complex(0, 0.25)]
]
Load = [
    [2, complex(0.5, 0.3)],
    [3, complex(1, 0.5)],
    [4, complex(1, 0.5)],
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'V':1, 'P':1.2 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 3 branches and 4 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.25j
    2  |    1 -  3 | 0.5j
    3  |    3 -  4 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-6.j -0.+4.j -0.+2.j  0.+0.j]
 [-0.+4.j  0.-4.j  0.+0.j  0.+0.j]
 [-0.+2.j  0.+0.j  0.-6.j -0.+4.j]
 [ 0.+0.j  0.+0.j -0.+4.j  0.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -0.5000   -0.3000
   3     PV   1.0000        ?    0.2000         ?
   4     PQ        ?        ?   -1.0000   -0.5000

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] + V₁V₄[G₁,₄cos𝜃₁,₄+B₁,₄sin𝜃₁,₄] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*4.0*sin(0-𝜃₂) + 1*1*2.0*sin(0-𝜃₃)
P₁ = -4.0000V₂sin(𝜃₂)-2.0000sin(𝜃₃).

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Three-bus radial network

We can also develop the equations for networks with complex impedances:

![Week02_2Bus_radial.png](Figures/Week02_2Bus_radial.png)

In [14]:
Connectivity = [
    [1, 2, complex(0.1, 0.2)],
    [2, 3, complex(0.1, 0.2)]
]
Load = [
    [2, complex(1.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1.05, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 2 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0.2j)
    2  |    2 -  3 | (0.1+0.2j)
_______|___________|__________

Ybus = 
 [[ 2.-4.j -2.+4.j  0.+0.j]
 [-2.+4.j  4.-8.j -2.+4.j]
 [ 0.+0.j -2.+4.j  2.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0500   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.5000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*[G₁,₁*cos𝜃₁,₁+B₁,₁*sin𝜃₁,₁] + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = V₁²*G₁,₁*cos𝜃₁,₁ + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = 1.05²*2.0*cos(0-0) + 1.05*V₂*[-2.0*cos(0-𝜃₂)+4.0*sin(0-𝜃₂)]
P₁ = 2.2050 + 1.0500V₂[-2.0000cos(𝜃₂)-4.0000sin(𝜃₂)].....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Create your own example

Try to create your own examples:

In [15]:
Connectivity = [
    [1, 2, complex(0.1, 0.2)],
    [2, 3, complex(0.1, 0.2)]
]
Load = [
    [2, complex(1.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1.05, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 2 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0.2j)
    2  |    2 -  3 | (0.1+0.2j)
_______|___________|__________

Ybus = 
 [[ 2.-4.j -2.+4.j  0.+0.j]
 [-2.+4.j  4.-8.j -2.+4.j]
 [ 0.+0.j -2.+4.j  2.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0500   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.5000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*[G₁,₁*cos𝜃₁,₁+B₁,₁*sin𝜃₁,₁] + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = V₁²*G₁,₁*cos𝜃₁,₁ + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = 1.05²*2.0*cos(0-0) + 1.05*V₂*[-2.0*cos(0-𝜃₂)+4.0*sin(0-𝜃₂)]
P₁ = 2.2050 + 1.0500V₂[-2.0000cos(𝜃₂)-4.0000sin(𝜃₂)].....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)